In [1]:
import pandas as pd
import numpy as np
import common

In [2]:
def calculate_changeover_cost():
    man_seq = pd.read_csv('manufacture_sequence.csv')
    lines=['A1','A2','B1','B2','B3','C1']
    changeover_costs = {'A1':33536,'A2':15947,'B1':13333,'B2':13333,'B3':13333,'C1':54353.39}
    total_changeover_cost=0
    for line in lines:
        plant = line[0]
        line_no = int(line[1])
        line_seq = man_seq[(man_seq.Plant==plant) & (man_seq.Line==line_no)]
        curr_prod = None
        changes=0
        for index,row in line_seq.iterrows():
            product = row.Product_ID
            if product != curr_prod and not pd.isnull(product):
                if curr_prod != None:
                    changes += 1
                curr_prod = product
        total_changeover_cost += changes*changeover_costs[line]
    return total_changeover_cost

In [3]:
def calculate_production_cost():
    man_seq = pd.read_csv('manufacture_sequence.csv')  
    pro_cos = pd.read_csv('production_cost.csv')
    pro_cap = pd.read_csv('production_capacity.csv')
    
    lines=['A1','A2','B1','B2','B3','C1']
    total_prod_cost=0
    for line in lines:
        plant=line[0]
        line_no = int(line[1])
        line_seq = man_seq[(man_seq.Plant==plant) & (man_seq.Line==line_no)]
        for month in range(37,40):
            month_seq = line_seq[(line_seq.Month==month)]
            for product,days in month_seq.groupby('Product_ID').Product_ID.count().iteritems():
                capacity = pro_cap[(pro_cap.Plant==plant)&
                    (pro_cap.Line==line_no) &(pro_cap.Product==product)].Capacity.values[0]
                pot_prod = capacity*days
                cost_per_mt = pro_cos[(pro_cos.Product_ID==product) &
                                     (pro_cos.Plant==plant)].Production_cost.values[0]
                total_prod_cost += pot_prod*cost_per_mt
    return total_prod_cost

In [4]:
def calculate_delivery_cost():
    shi_reg = pd.read_csv('shipping_region.csv')
    del_cos = pd.read_csv('delivery_cost.csv')
    regions=[]
    for i in range(1,19):
        regions.append('R'+str(i))
    products=[]
    for i in range(1,82):
        products.append('P'+str(i))
    total_delivery_cost=0
    for plant in ['A','B','C']:
        for region in regions:
            region_shipments = shi_reg[(shi_reg.Plant==plant) &
                (shi_reg.Region==region)].Shipping_to_region_quantity.sum()
            del_cost = del_cos[(del_cos.Plant==plant) &
                              (del_cos.Region==region)].Delivery_cost.values[0]
            total_delivery_cost += (region_shipments*del_cost)
    return total_delivery_cost

In [5]:
def calculate_cost():
    fixed_cost=(33536+15947+13333+13333+13333+54353.39)*90
    changeover_cost=calculate_changeover_cost()
    production_cost=calculate_production_cost()
    delivery_cost=calculate_delivery_cost()
    return fixed_cost+changeover_cost+production_cost+delivery_cost

In [6]:
def calculate_revenue():
    dem_pri = pd.read_csv('demand_price.csv')
    shi_reg = pd.read_csv('shipping_region.csv')
    dem_for = pd.read_csv('demand_forecast.csv')
    regions=[]
    for i in range(1,19):
        regions.append('R'+str(i))
    products=[]
    for i in range(1,82):
        products.append('P'+str(i))
    total_revenue=0
    for region in regions:
        for product in products:
            dem_price = dem_pri[(dem_pri.Product_ID==product) &
                               (dem_pri.Region==region)].Demand_price.values[0]
            act_quantity = shi_reg[(shi_reg.Region==region)&
                                  (shi_reg.Product_ID==product)].Shipping_to_region_quantity.sum()
            dem_quantity = dem_for[(dem_for.Region==region)&(dem_for.Product_ID==product)].Demand.sum()
            total_revenue += dem_price*(min(act_quantity,dem_quantity))
    return total_revenue

In [8]:
def get_margin_percent():
    total_revenue = calculate_revenue()
    total_cost = calculate_cost()
    print("Revenue-",total_revenue,'Cost-',total_cost)
    return ((total_revenue-total_cost)*100)/total_cost
get_margin_percent()

Revenue- 80404973.9895 Cost- 73837663.2627


8.8942559075564187

In [21]:
def list_best_product_for_lines():
    pro_cos = pd.read_csv('production_cost.csv')
    pro_cap = pd.read_csv('production_capacity.csv')
    dem_pri = pd.read_csv('demand_price.csv')
    del_cos = pd.read_csv('delivery_cost.csv')
    products = []
    for i in range(1,82):
        products.append('P'+str(i))
    lines=['A1','A2','B1','B2','B3','C1']
    regions=[]
    for i in range(1,19):
        regions.append('R'+str(i))
    profits =[]
    for line in lines:
        plant = line[0]
        line_no = int(line[1])
        for product in products:
            prod_cost = pro_cos[(pro_cos.Product_ID==product) &
                               (pro_cos.Plant==plant)].Production_cost.values[0]
            prod_cap = pro_cap[(pro_cap.Plant==plant)&
                        (pro_cap.Line==line_no) & (pro_cap.Product==product)].Capacity.values[0]
            for region in regions:
                prod_price = dem_pri[(dem_pri.Product_ID==product) &
                                    (dem_pri.Region==region)].Demand_price.values[0]
                del_cost = del_cos[(del_cos.Plant==plant) &
                                  (del_cos.Region==region)].Delivery_cost.values[0]
                if prod_price > 0:
                    profit = prod_cap*(prod_price-(prod_cost+del_cost))
                else:
                    profit = 0
                tag = line+'-'+product+'-'+region+'-'+str(prod_cap)
                profits.append({'tag':tag,'profit':profit})
    profits.sort(key=lambda x:x['profit'],reverse=True)
    return profits

In [22]:
def generate_best_files():
    best_strs = ['A1-P61-R8-75','A2-P43-R18-130','B1-P19-R4-205','B2-P47-R18-127','B3-P61-R8-75','C1-P34-R7-181']
    best=[]
    for pstr in best_strs:
        tokens = pstr.split('-')
        best_entry = {'Line':tokens[0],'Product':tokens[1],'Region':tokens[2],'Capacity':float(tokens[3])}
        best.append(best_entry)
    products = []
    for i in range(1,82):
        products.append('P'+str(i))    
    regions=[]
    for i in range(1,19):
        regions.append('R'+str(i))   
    
    # demand_forecast
    records=[]
    for product in products:
        for region in regions:
            for month in range(37,40):
                records.append([product,region,month,0])
    df = pd.DataFrame(records)  
    df.columns=['Product_ID','Region','Month','Demand']
    for be in best:
        for month in range(37,40):
            df.loc[(df.Product_ID==be['Product']) &
                  (df.Region==be['Region']) &
                   (df.Month==month),'Demand']+=be['Capacity']*28
    df = df.sort_values(['Product_ID','Region','Month'])
    df.to_csv('demand_forecast.csv',index=False)
    
    # manufacturing sequence
    plants=['A','B','C']
    line_nos={'A':[1,2],'B':[1,2,3],'C':[1]}
    records=[]
    for plant in plants:
        for line_no in line_nos[plant]:
            for month in range(37,40):
                for day in range(1,31):
                    records.append([plant,line_no,month,day,''])
    df = pd.DataFrame(records)
    df.columns=['Plant','Line','Month','Day','Product_ID']
    for be in best:
        plant = be['Line'][0]
        line_no = int(be['Line'][1])
        for month in range(37,40):
            for day in range(1,15):
                df.loc[(df.Plant==plant)&(df.Line==line_no)&
                      (df.Month==month)&(df.Day==day),'Product_ID']=be['Product']
            for day in range(16,30):
                df.loc[(df.Plant==plant)&(df.Line==line_no)&
                      (df.Month==month)&(df.Day==day),'Product_ID']=be['Product']
    df.to_csv('manufacture_sequence.csv',index=False)
    

    # shipping_region
    records=[]
    for plant in plants:
        for region in regions:
            for month in range(37,40):
                for product in products:
                    records.append([plant,region,product,month,0])
    df = pd.DataFrame(records)
    df.columns=['Plant','Region','Product_ID','Month','Shipping_to_region_quantity']
    for be in best:
        plant = be['Line'][0]
        for month in range(37,40):
            df.loc[(df.Plant==plant)&(df.Region==be['Region'])&
                    (df.Product_ID==be['Product']) & 
                   (df.Month==month),'Shipping_to_region_quantity'] += be['Capacity']*28
    df.to_csv('shipping_region.csv',index=False)    